In [5]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import skfuzzy as fuzz
from sklearn.metrics import fowlkes_mallows_score
import math

# wczytanie danych

In [224]:
pd.options.display.max_rows = 6

ratings = pd.read_csv(Path('..','..','res','data','ratings.csv'))
users = pd.read_csv(Path('..','..','res','data','users.csv'))
movies = pd.read_csv(Path('..','..','res','data','movies.csv'))


# zmiana napisów na liczby
label_encoder = LabelEncoder()
users['gender'] = label_encoder.fit_transform(users['gender'])
users['occupation'] = label_encoder.fit_transform(users['occupation'])


matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')
matrix = matrix.fillna(0)

train_set, test_set = train_test_split(matrix, test_size=0.2)#, random_state=42)

users_train = users[~users['user_id'].isin(test_set.index.to_list())]
users_test = users[users['user_id'].isin(test_set.index.to_list())]

items_train = ratings[~ratings['user_id'].isin(test_set.index.to_list())]
#jeżeli jakiś przedmiot wpadnie tylko do zbioru testowego to "wypada" z dalszych analiz
#ponieważ ma przed wejściem zimnych userów sume ratingów = 0, więc zgodnie z moją metodą
#nie zostanie zarekomendowany w topN filmów
items_test = ratings[ratings['user_id'].isin(test_set.index.to_list())]

train_set.to_csv(Path('..','..','outputs','train_set.csv'), index = False)
test_set.to_csv(Path('..','..','outputs','test_set.csv'), index = False)

all_users_clustered = users_train_clusters = cluster_users(users_train.copy(), users_test.copy())
users_train_clustered = all_users_clustered[0]
users_test_clustered = all_users_clustered[1]
#items_clustered = cluster_items(items_train)
#items_clustered.to_csv(Path('..','..','outputs','items_clustered3.csv'), index=False)
items_clustered = items_train.copy()
items_clustered['cluster'] = 1


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 25, cost: 1165.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 57, cost: 1194.0
Run 2, iteration: 2/100, moves: 9, cost: 1194.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 31, cost: 1235.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 20, cost: 1197.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 50, cost: 1253.0
Run 5, iteration: 2/100, moves: 0, cost: 1253.0
Best run was number 1


In [225]:
def cluster_items(items_info):
    data_to_cluster = items_info['rating'].to_numpy()
    data_to_cluster = data_to_cluster.reshape(-1,1)
    K_FCM = 100
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data_to_cluster.T, K_FCM, 2, error=0.005, maxiter=1000)
    clusters = np.argmax(u, axis=0)
    items_info['cluster'] = clusters
    return(items_info)


def cluster_items2(items_info):
    data_to_cluster = items_info[['rating', 'user_id', 'movie_id']].to_numpy()
    print(items_info.shape, data_to_cluster.shape)
    data_to_cluster = data_to_cluster.reshape(-1,3)
    print(data_to_cluster.shape)
    K_FCM = 100
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data_to_cluster.T, K_FCM, 2, error=0.005, maxiter=1000)
    clusters = np.argmax(u, axis=0)
    print(len(clusters))
    items_info['cluster'] = clusters
    return(items_info)

"""cluster_items_3cols = cluster_items2(items_test)"""

'cluster_items_3cols = cluster_items2(items_test)'

In [226]:
def cluster_users(users_info, test_users_info):
    data_to_cluseter = users_info[['age','gender','occupation']]
    kmode = KModes(n_clusters=5, init = "random", n_init = 5, verbose=1, random_state=42) 
    clusters = kmode.fit_predict(data_to_cluseter) 
    users_info['cluster'] = clusters
    data_to_cluster_test = test_users_info[['age','gender','occupation']]
    test_clusters = kmode.predict(data_to_cluster_test)
    test_users_info['cluster'] = test_clusters
    return(users_info, test_users_info)

In [282]:
def IP(train_matrix, items_clustered):
    train_numpy = train_matrix.to_numpy()
    #szukanie IPmax
    IP = np.sum(train_numpy, axis = 0) #sumowanie po kolumnach
    IP = IP.reshape(1,-1)
    IPmax = np.amax(IP)
    IPmax_coordinates = np.argmax(IP) + 1 #bo filmy mają indeksy od 1
    item_rating = pd.DataFrame({'rating':IP[0], 'movie_id': train_matrix.columns.to_list()})
    
    #znalezienie filmu o najwyższym IP
    IPmax_movie_id = item_rating[item_rating['rating'] == IPmax]['movie_id'].values[0]
    
    #znalezienie clustra, gdzie znajduje się ten film i pozostałych należących tu filmów
    IPmax_cluster = items_clustered[items_clustered['movie_id'] == IPmax_movie_id].iloc[0,4]
    max_clust_movies = items_clustered[items_clustered['cluster'] == IPmax_cluster]['movie_id'].to_list()
    max_clust_movies = list(set(max_clust_movies))

    #obliczenie IP^
    IP_dash = IP[:,np.array(max_clust_movies) - 1] / IPmax
    return(IP_dash, max_clust_movies)

IP_dash_movies = IP(train_set, items_clustered)
IP_dashhhh = IP_dash_movies[0]
IP_moviesss = IP_dash_movies[1]

items clustered        user_id  movie_id  rating  timestamp  cluster
0          196       242       3  881250949        1
1          186       302       3  891717742        1
2           22       377       1  878887116        1
...        ...       ...     ...        ...      ...
99997      276      1090       1  874795795        1
99998       13       225       2  882399156        1
99999       12       203       3  879959583        1

[81045 rows x 5 columns]
train_numpy [[3. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [2. 3. 2. ... 0. 0. 0.]
 ...
 [3. 0. 0. ... 0. 0. 0.]
 [4. 0. 3. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]


In [269]:
#def UP_HP(train_test_set, cluster, clust_data, IP_dash, alpha, selected_movies, ratings_matrix):
def UP_HP(train_matrix, test_matrix, test_users_cl, train_users_cl, cluster, IP_dash, alpha, selected_movies, ratings_matrix):    
    
    #utworzenie macierzy, gdzie ratingi dane przez testowych są zerami
    test_set_zero = test_matrix.copy()
    test_set_zero[:] = 0 
    train_and_test = pd.concat([train_matrix, test_set_zero], ignore_index=False)
    train_and_test = train_and_test.sort_index()
    
    concated_users_clustered = pd.concat([train_users_cl,test_users_cl], ignore_index=True).sort_values('user_id')

    #znalezienie użytkowników w danym clustrze
    users_in_cluster = concated_users_clustered[concated_users_clustered['cluster'] == cluster]['user_id'].to_list()
    users_in_cluster = list(set(users_in_cluster))
    
    #macierz ratingów tylko dla tych userów i filmów z clustra z IPmax
    ratings_matrix_cluster = train_and_test.loc[train_and_test.index.isin(users_in_cluster),selected_movies].sort_index()
    ratings_matrix_cluster_np = ratings_matrix_cluster.to_numpy()

    #obliczanie user popularity
    UP = np.sum(ratings_matrix_cluster_np, axis=1) ##sumowanie po wierszach
    UP = UP.reshape(-1,1)
    UPmax = np.amax(UP)
    UPmax_coordinates = np.argmax(UP)
    UP_dash = UP / UPmax
    
    # obliczenie UIP i hybrydy
    UIP = UP_dash / IP_dash
    #wstawienie zer tam gdzie w rzeczywistości uzytkownik treningowy nie ocenił danego filmu
    UIP = np.where(ratings_matrix_cluster == 0, 0, UIP)
    HP = alpha * IP_dash + (1-alpha) * np.sum(UIP,axis = 0).reshape(1,-1)
    return(HP)
    #return np.sum(pd.DataFrame(UIP, index=ratings_matrix_cluster.index, columns=ratings_matrix_cluster.columns).loc[243,9])

"""test_set_zero = test_set.copy()
test_set_zero[:] = 0
concated_users_clustered = pd.concat([us_tr_cl,us_te_cl], ignore_index=True).sort_values('user_id')
train_and_test = pd.concat([train_set, test_set_zero], ignore_index=False)
train_and_test = train_and_test.sort_index()"""

UP_HP(train_set, test_set, users_test_clustered, users_train_clustered, 1, IP_dashhhh, 0.5, IP_moviesss, matrix).shape

[[6.46762881e+00 5.87551097e+00 1.33723953e+01 ... 4.92125984e-04
  7.38188976e-04 7.38188976e-04]]


(1, 1673)

In [229]:
def HP_for_clust(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl):

    #zapewnienie prawdiłowej kolejnosci wierszy i kolumn
    train_matrix = train_matrix.sort_index()
    test_matrix = test_matrix.sort_index()
    
    #podział na clustry
    #users_clustered = cluster_users(users_train.copy())
    #items_clustered = cluster_items(items_train.copy())
    
    """users_clustered = pd.read_csv(Path('..','..','outputs','users_train_clustered.csv'))
    #items_clustered = pd.read_csv(Path('..','..','outputs','items_train_clustered.csv'))
    users_clustered['cluster'] = 1
    #przypisanie nowych użytkowników do clustrów
    kmode = KModes(n_clusters=5, init = "random", n_init = 5, random_state = 42)#verbose=1, random_state=42) 
    train_clusters = kmode.fit_predict(users_train)
    test_users_clusters = kmode.predict(users_test)
    test_users_clustered = users_test.copy() ##OCEŃ CZY KOPIOWANIE JEST KONIECZNE
    test_users_clustered['cluster'] = test_users_clusters"""
    
    ### klastrowanie ###
    test_users_clustered = test_users_cl
    test_users_clusters = test_users_clustered['cluster'].values
    items_clustered = items_cl
    
    ## IP ##
    IP_results = IP(train_matrix, items_clustered)
    IP_dash = IP_results[0]
    movies_from_max_cluster = IP_results[1]

    clust_HP = {}
    for cl in list(set(test_users_clusters)):
        HP = UP_HP(train_matrix, test_matrix, test_users_clustered, train_users_cl, cl, IP_dash, alpha, movies_from_max_cluster, ratings_matrix)[0].tolist()  
        clust_HP[cl] = (movies_from_max_cluster, HP)
    return(clust_HP, test_users_clustered) #słownik [cluster[]: [filmy z clustra gdzie IP_max, HP],
                                            #testowi uzytkownicy z wygenerowanymi klastrami

"""z = HP_for_clust(test_matrix, train_matrix, 0.5, ratings_matrix, us_te_cl, us_tr_cl, items_clusters)
len(z[0][1][0]) #długość movies_from_max_cluster
z[0][1][1][-3] #3 od konca wartość z HP"""

'z = HP_for_clust(test_matrix, train_matrix, 0.5, ratings_matrix, us_te_cl, us_tr_cl, items_clusters)\nlen(z[0][1][0]) #długość movies_from_max_cluster\nz[0][1][1][-3] #3 od konca wartość z HP'

In [230]:
def create_matrix_for_generated(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl):
    HP_and_clust = HP_for_clust(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl)
    movies_and_HP = HP_and_clust[0] #słownik [cluster]:[filmy do polecenia, HP]
    
    #dataframe testowych uzytkownikow z przypisanymi clustrami
    test_users_with_clusters = HP_and_clust[1]
    #dataframe pokazujący cluster i ilu uzytkowników do niego nalezy
    num_users_in_clust = test_users_with_clusters.groupby('cluster').agg(users_number = ('cluster','size')).reset_index()
    num_users_in_clust_dict = num_users_in_clust.set_index('cluster')['users_number'].to_dict() #liczba użytkowników w każdym clustrze

    #słownik cluster: id uzytkownikow którzy do niego nalezą
    users_per_cluster = test_users_with_clusters.groupby('cluster')['user_id'].apply(list).to_dict() #uzytkownicy (id) w kazdym clustrze

    test_users_dictionary = {'user_id':[],
                         'movie_id':[],
                         'rating':[]}
    
    for cl in users_per_cluster.keys():
        num_of_users = num_users_in_clust_dict[cl]
        num_of_items = len(movies_and_HP[cl][0])

        multiplied_users = users_per_cluster[cl] * num_of_items
        multiplied_users.sort()
        test_users_dictionary['user_id'].extend(multiplied_users)
        #aktualizuję uzytkowników w clustrze powielając ich tyle razy ile jest przedmiotów, 
        #bo każdy musi dany przedmiot mieć oceniony
        users_per_cluster[cl] = multiplied_users

        #powielam przedmioty i oceny tyle razy ilu jest użytkowników
        movies_and_HP[cl] = (movies_and_HP[cl][0] * num_of_users, movies_and_HP[cl][1] * num_of_users)
        
        test_users_dictionary['movie_id'].extend(movies_and_HP[cl][0])
        test_users_dictionary['rating'].extend(movies_and_HP[cl][1])

        test_ratings_HP = pd.DataFrame(test_users_dictionary)
        test_ratings_HP.to_csv(Path('..','..','outputs','test_ratings.csv'), index=False)
        test_ratings_matrix = test_ratings_HP.pivot(index='user_id', columns='movie_id', values='rating')
        test_ratings_matrix = test_ratings_matrix.fillna(0)

    return test_ratings_matrix

create_matrix_for_generated(test_set, train_set, 0.5, matrix, users_test_clustered, users_train_clustered, items_clustered)


movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
6,33.876501,61.861183,51.608481,56.955898,52.749747,40.530260,36.117754,46.167017,40.146646,49.265176,...,0.000738,129.544163,153.227271,229.840292,0.000738,0.000246,0.000738,0.000492,228.674914,0.000738
17,9.739352,28.990976,17.081301,14.903999,17.688430,0.019931,12.985426,8.716253,8.892647,11.397785,...,0.000738,0.000984,0.000738,0.000492,0.000738,0.000246,0.000738,0.000492,0.000738,0.000738
24,17.921470,24.598677,22.244171,26.697778,40.876049,40.609053,16.244004,26.618142,22.143779,27.140124,...,71.916401,0.000984,0.000738,0.000492,0.000738,0.000246,0.000738,0.000492,0.000738,0.000738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,17.921470,24.598677,22.244171,26.697778,40.876049,40.609053,16.244004,26.618142,22.143779,27.140124,...,71.916401,0.000984,0.000738,0.000492,0.000738,0.000246,0.000738,0.000492,0.000738,0.000738
942,17.921470,24.598677,22.244171,26.697778,40.876049,40.609053,16.244004,26.618142,22.143779,27.140124,...,71.916401,0.000984,0.000738,0.000492,0.000738,0.000246,0.000738,0.000492,0.000738,0.000738
943,14.294266,38.781307,44.717724,24.243913,28.198769,16.462367,15.861163,16.984743,10.743998,13.722114,...,0.000738,0.000984,0.000738,0.000492,0.000738,189.049828,63.017266,94.525283,0.000738,201.314016


In [231]:
def show_Hu(row):
    #wybieranie id filmów, dla których w zbiorze testowym użytkownik wystawił jakieś oceny
    nonzero = row[row > 0].index
    nonzero_indices = nonzero.values
    row['Hu'] = nonzero_indices.tolist()
    return row

In [232]:
def find_topN_items(users_ratings,N):
    topN = users_ratings.nlargest(N) #N najlepiej ocenianych filmów (rating i indeks) 
                                        #zwraca się od najwiekszej do najmn wartosci
    topN_indices = topN.index #indeksy tych najlepiej ocenianych
    #wyrzucam z topN filmy, dla których wystawiono zero
    top_nonzero_indices = users_ratings[topN_indices][users_ratings[topN_indices] > 0].index
    users_ratings["topN_movies"] = top_nonzero_indices.values
    return users_ratings

In [233]:
def NDCG(row,N):
    generated = row.loc['topN_movies']
    from_test_set = row.loc['Hu']
    #sprawdzenie który n-ty element z TopN został irl oceniony przez użytkownika
    available_n = [index + 1  for index, element in enumerate(generated) if element in from_test_set]
    DCG = sum([1 / math.log(1+n,2) for n in available_n])
    IDCG = sum([math.log(1+n,2) for n in range(1,N+1)])
    NDCG = DCG / IDCG
    row['NDCG'] = NDCG
    return row

In [234]:
def Precision_Recall(row,N):
    generated = row.loc['topN_movies']
    from_test_set = row.loc['Hu']
    intersection = sum([1 for element in generated if element in from_test_set])
    
    Precision = intersection * 100 / N
    row['Precision'] = Precision
    
    row['Recall'] = intersection * 100 / len(from_test_set)
    return row

In [279]:
def nazwepozniej(test_matrix, train_matrix, alpha, N, ratings_matrix, test_users_cl, train_users_cl, items_cl):
    
    generated = create_matrix_for_generated(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl)
    test_with_Hu = test_matrix.apply(lambda row: show_Hu(row), axis = 1)
    generated_topN = generated.apply(lambda row: find_topN_items(row, N), axis = 1)
    Hu = test_with_Hu.iloc[:,[-1]].sort_index()
    
    merged = pd.merge(generated_topN, Hu, on='user_id', how = 'left')
    merged = merged.apply(lambda row:  NDCG(row,N),axis = 1)
    merged = merged.apply(lambda row:  Precision_Recall(row,N),axis = 1)
    return(merged)
output_final = nazwepozniej(test_set, train_set, 0.5, 50, matrix, users_test_clustered, users_train_clustered, items_clustered)
"""hu1 = output_final[0]
generated1 = output_final[1]
test_with_hu1 = output_final[2]
generatetopN1 = output_final[3]
merged1 = output_final[4]"""

items clustered        user_id  movie_id  rating  timestamp  cluster
0          196       242       3  881250949        1
1          186       302       3  891717742        1
2           22       377       1  878887116        1
...        ...       ...     ...        ...      ...
99997      276      1090       1  874795795        1
99998       13       225       2  882399156        1
99999       12       203       3  879959583        1

[81045 rows x 5 columns]
[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [5. 0. 0. ... 0. 0. 0.]]
[[1.79214698e+01 2.45986768e+01 2.22441712e+01 ... 4.92125984e-04
  7.38188976e-04 7.38188976e-04]]
[[6.46762881e+00 5.87551097e+00 1.33723953e+01 ... 4.92125984e-04
  7.38188976e-04 7.38188976e-04]]
[[3.38765014e+01 6.18611833e+01 5.16084808e+01 ... 4.92125984e-04
  2.28674914e+02 7.38188976e-04]]
[[9.73935162e+00 2.89909755e+01 1.70813012e+01 ... 4.92125984e-04
  7.38188976

'hu1 = output_final[0]\ngenerated1 = output_final[1]\ntest_with_hu1 = output_final[2]\ngeneratetopN1 = output_final[3]\nmerged1 = output_final[4]'

In [262]:
output_final

movie_id,1,2,3,4,5,6,7,8,9,10,...,1678,1679,1680,1681,1682,topN_movies,Hu,NDCG,Precision,Recall
user_id,,,,,,,,,,,,,,,,,,,,,
6,33.876501,61.861183,51.608481,56.955898,52.749747,40.530260,36.117754,46.167017,40.146646,49.265176,...,0.000246,0.000738,0.000492,228.674914,0.000738,"[599, 1659, 1626, 1453, 1461, 1320, 1325, 1329...","[1, 7, 8, 9, 12, 13, 14, 15, 19, 21, 22, 23, 2...",0.0,0.0,0.0
17,9.739352,28.990976,17.081301,14.903999,17.688430,0.019931,12.985426,8.716253,8.892647,11.397785,...,0.000246,0.000738,0.000492,0.000738,0.000738,"[1486, 1493, 1494, 1621, 1250, 1655, 1440, 149...","[1, 7, 9, 13, 100, 111, 117, 125, 126, 137, 15...",0.0,0.0,0.0
24,17.921470,24.598677,22.244171,26.697778,40.876049,40.609053,16.244004,26.618142,22.143779,27.140124,...,0.000246,0.000738,0.000492,0.000738,0.000738,"[830, 852, 784, 1546, 1548, 1551, 1557, 1559, ...","[7, 8, 9, 11, 12, 25, 41, 55, 56, 58, 64, 69, ...",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,17.921470,24.598677,22.244171,26.697778,40.876049,40.609053,16.244004,26.618142,22.143779,27.140124,...,0.000246,0.000738,0.000492,0.000738,0.000738,"[830, 852, 784, 1546, 1548, 1551, 1557, 1559, ...","[9, 14, 19, 50, 93, 100, 116, 124, 126, 137, 2...",0.0,0.0,0.0
942,17.921470,24.598677,22.244171,26.697778,40.876049,40.609053,16.244004,26.618142,22.143779,27.140124,...,0.000246,0.000738,0.000492,0.000738,0.000738,"[830, 852, 784, 1546, 1548, 1551, 1557, 1559, ...","[31, 50, 71, 79, 95, 97, 99, 117, 124, 131, 13...",0.0,0.0,0.0
943,14.294266,38.781307,44.717724,24.243913,28.198769,16.462367,15.861163,16.984743,10.743998,13.722114,...,189.049828,63.017266,94.525283,0.000738,201.314016,"[1502, 1510, 1482, 1682, 1678, 1426, 104, 1145...","[2, 9, 11, 12, 22, 23, 24, 27, 28, 31, 38, 41,...",0.0,0.0,0.0


0

In [290]:
def main(ratings_data, items_data, users_data, alpha, N):

    matrix = ratings_data.pivot(index='user_id', columns='movie_id', values='rating')
    matrix = matrix.fillna(0)
    
    train_set, test_set = train_test_split(matrix, test_size=0.2, random_state=42)

    users_train = users_data[~users_data['user_id'].isin(test_set.index.to_list())]
    users_test = users_data[users_data['user_id'].isin(test_set.index.to_list())]

    items_train = ratings_data[~ratings_data['user_id'].isin(test_set.index.to_list())]
    #jeżeli jakiś przedmiot wpadnie tylko do zbioru testowego to "wypada" z dalszych analiz
    #ponieważ ma przed wejściem zimnych userów sume ratingów = 0, więc zgodnie z moją metodą
    #nie zostanie zarekomendowany w topN filmów
    items_test = ratings_data[ratings_data['user_id'].isin(test_set.index.to_list())]

    train_set.to_csv(Path('..','..','outputs','train_set.csv'), index = False)
    test_set.to_csv(Path('..','..','outputs','test_set.csv'), index = False)

    all_users_clustered = users_train_clusters = cluster_users(users_train.copy(), users_test.copy())
    users_train_clustered = all_users_clustered[0]
    users_test_clustered = all_users_clustered[1]
    items_clustered = items_train.copy()
    items_clustered['cluster'] = 1
    final_matrix = nazwepozniej(test_set, train_set, alpha, N, matrix, users_test_clustered, users_train_clustered, items_clustered)
    return(final_matrix)

In [292]:
main(ratings, movies, users, 0.5, 20)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 78, cost: 1151.0
Run 1, iteration: 2/100, moves: 4, cost: 1151.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 63, cost: 1162.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 25, cost: 1186.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 351, cost: 1194.0
Run 4, iteration: 2/100, moves: 5, cost: 1194.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 11, cost: 1218.0
Run 5, iteration: 2/100, moves: 1, cost: 1218.0
Best run was number 1
items clustered        user_id  movie_id  rating  timestamp  cluster
0          196       242       3  881250949        1
1          186       302       3  891717742        1
2           2

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1681,1682,topN_movies,Hu,NDCG,Precision,Recall
user_id,,,,,,,,,,,,,,,,,,,,,
24,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[7, 8, 9, 11, 12, 25, 41, 55, 56, 58, 64, 69, ...",0.0,0.0,0.0
31,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,237.266163,0.000740,137.827871,243.297604,0.00074,"[1621, 1659, 1290, 1626, 104, 1453, 1461, 1502...","[32, 79, 124, 135, 136, 153, 175, 192, 262, 26...",0.0,0.0,0.0
32,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[7, 9, 50, 100, 111, 117, 118, 122, 151, 181, ...",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[1, 7, 9, 15, 25, 50, 100, 105, 106, 111, 117,...",0.0,0.0,0.0
939,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[9, 15, 106, 118, 121, 127, 220, 222, 237, 252...",0.0,0.0,0.0
941,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,237.266163,0.000740,137.827871,243.297604,0.00074,"[1621, 1659, 1290, 1626, 104, 1453, 1461, 1502...","[1, 7, 15, 117, 124, 147, 181, 222, 257, 258, ...",0.0,0.0,0.0


In [289]:
def main_article():
    article_data = pd.read_csv(Path('..','..','res','data','article.csv'))
    rating_matrix_article = article_data.pivot(index = 'user_id', columns='movie_id', values='rating')
    matrix_train = rating_matrix_article[rating_matrix_article.index.isin([1,2,3])]
    matrix_test = rating_matrix_article[rating_matrix_article.index.isin([4])]
    article_items_cl = article_data[article_data['user_id'].isin([1,2,3])]
    article_items_cl['cluster'] = 1
    article_users_cl = pd.DataFrame({'user_id':[1,2,3],
                                    'cluster':[1,1,1]})
    article_users_test_cl1 = pd.DataFrame({'user_id':[4],
                                    'cluster':[1]})
    final_matrix = nazwepozniej(matrix_test, matrix_train, 0.5, 20, rating_matrix_article, article_users_test_cl1, article_users_cl, article_items_cl)
    return(final_matrix)
article = main_article()

items clustered     user_id  movie_id  rating  timestamp  cluster
0         1         1       3         77        1
1         1         2       4         77        1
2         1         3       0         77        1
..      ...       ...     ...        ...      ...
18        3         5       2         77        1
19        3         6       4         77        1
20        3         7       3         77        1

[21 rows x 5 columns]
train_numpy [[3 4 0 5 2 0 0]
 [0 5 5 4 0 3 3]
 [2 3 3 0 2 4 3]]
[[2.06833333 1.775      1.72083333 1.50833333 2.49166667 1.87738095
  2.1       ]]


C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\2553056881.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article_items_cl['cluster'] = 1


ZeroDivisionError: division by zero

In [265]:
merged_Hu

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1681,1682,topN_movies,Hu,NDCG,Precision,Recall
user_id,,,,,,,,,,,,,,,,,,,,,
24,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[7, 8, 9, 11, 12, 25, 41, 55, 56, 58, 64, 69, ...",0.0,0.0,0.0
31,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,237.266163,0.000740,137.827871,243.297604,0.00074,"[1621, 1659, 1290, 1626, 104, 1453, 1461, 1502...","[32, 79, 124, 135, 136, 153, 175, 192, 262, 26...",0.0,0.0,0.0
32,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[7, 9, 50, 100, 111, 117, 118, 122, 151, 181, ...",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[1, 7, 9, 15, 25, 50, 100, 105, 106, 111, 117,...",0.0,0.0,0.0
939,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,0.000986,75.408967,0.000986,0.000740,0.00074,"[1661, 1550, 957, 1517, 1539, 1509, 1663, 1665...","[9, 15, 106, 118, 121, 127, 220, 222, 237, 252...",0.0,0.0,0.0
941,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,237.266163,0.000740,137.827871,243.297604,0.00074,"[1621, 1659, 1290, 1626, 104, 1453, 1461, 1502...","[1, 7, 15, 117, 124, 147, 181, 222, 257, 258, ...",0.0,0.0,0.0


### do uruchomienia, jeśli chce się na sztywno clustry przypisać

In [19]:
us_te_cl = users_test.copy()
us_te_cl['cluster'] = 1

us_tr_cl = users_train.copy()
us_tr_cl['cluster'] = 1

it_cl = items_train.copy()
it_cl['cluster'] = 1

"""it_cl = article_items_cl
    it_cl"""

NameError: name 'users_test' is not defined

In [266]:
article_data = pd.read_csv(Path('..','..','res','data','article.csv'))
rating_matrix_article = article_data.pivot(index = 'user_id', columns='movie_id', values='rating')
matrix_train = rating_matrix_article[rating_matrix_article.index.isin([1,2,3])]
matrix_test = rating_matrix_article[rating_matrix_article.index.isin([4])]
train_numpy = matrix_train.to_numpy()
article_items_cl = article_data[article_data['user_id'].isin([1,2,3])]
article_items_cl['cluster'] = 1
article_users_cl = pd.DataFrame({'user_id':[1,2,3],
                                'cluster':[1,1,1]})
article_users_test_cl1 = pd.DataFrame({'user_id':[4],
                                'cluster':[1]})

C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1047447764.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article_items_cl['cluster'] = 1
